# Agentic RAG: Turbocharge Our RAG with Query Reformulation and Self-Query

RAG is using an LLM to answer a user query , but basing the answer on information retrieved from a knowledge base. It allows to ground the answer on true facts and reduce hallucinations, it allows to provide the LLM with domain-specific knowledge, and it allows fine-grained control of access to information from the knowledge base.

However, vanilla RAG has limitations:
- It **performs only one retrieval step**: if the results are bad, the generation in turn will be bad.
- **Semantic similarity is computed with the *user query* as a reference**, which may be suboptimal: for instance, the user query will often be a question and the document containing the true answer will be in affirmative voice, so its similarity score will be downgraded compared to other source documents in the interrogative form, leading to a risk of missing the relevant information.

We can alleviate these problems by making a **RAG Agent**.

In this example, we will bulid an agent which will formulate the query itself and critique to re-retrieve if needed.

Hence, it should naively recover some advanced RAG techniques:
- Instead of directly using the user query as the reference in semantic search, the agent formulates itself a reference sentence that can be closer to the targeted document, as in the paper [*Precise Zero-Shot Dense Retrieval without Relevance Labels*](https://huggingface.co/papers/2212.10496).
- The agent can be generated snippets and re-retrieve if needed, as in[Self-Query](https://docs.llamaindex.ai/en/stable/examples/evaluation/RetryQuery/)

## Setups

In [ ]:
!pip install -qU pandas langchain langchain-community sentence-transformers faiss-cpu smolagents transformers datasets

## Load dataset

In [1]:
import datasets

knowledge_base = datasets.load_dataset('m-ric/huggingface_doc', split='train')

In [2]:
knowledge_base

Dataset({
    features: ['text', 'source'],
    num_rows: 2647
})

In [3]:
knowledge_base[0]

{'text': ' Create an Endpoint\n\nAfter your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. \n\n## 1. Enter the Hugging Face Repository ID and your desired endpoint name:\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_repository.png" alt="select repository" />\n\n## 2. Select your Cloud Provider and region. Initially, only AWS will be available as a Cloud Provider with the `us-east-1` and `eu-west-1` regions. We will add Azure soon, and if you need to test Endpoints with other Cloud Providers or regions, please let us know.\n\n<img src="https://raw.githubusercontent.com/huggingface/hf-endpoints-documentation/main/assets/1_region.png" alt="select region" />\n\n## 3. Defi

Now we will prepare the knowledge base by processing the dataset and storing it into a vector database to be used by the retriever. We will use **LangChain** for its excellent vector database utilities. For the embedding model, we use [`thenlper/gte-small`](https://huggingface.co/thenlper/gte-small)

In [4]:
from tqdm import tqdm
from transformers import AutoTokenizer
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

In [ ]:
source_docs = [
    Document(page_content=doc['text'], metadata={'source': doc['source'].split('/')[1]})
    for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained('thenlper/gte-small'),
    chunk_size=200,
    chunk_overlap=20,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""]
)

embedding_model = HuggingFaceEmbeddings(model_name='thenlper/gte-small')

In [8]:
# Split docs and keep only unique ones
print("Splitting documents...")
docs_processed = []
unique_texts = {}
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in unique_texts:
            unique_texts[new_doc.page_content] = True
            docs_processed.append(new_doc)
print(f"Documents splitted into {len(docs_processed)} chunks")

Splitting documents...


100%|██████████| 2647/2647 [02:47<00:00, 15.80it/s]

Documents splitted into 39785 chunks


In [ ]:
print("Embedding documents...")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model,
    distance_strategy=DistanceStrategy.COSINE
)
print('Done!')

Embedding documents...


## Agentic RAG

We only need a `RetrieverTool` that our agent can leverage to retrieve information from the knowledge base.

Since we need to add a `vectordb` as an attribute of the tool, we cannot simply use the `simple_tool_constructor` with a `@tool` decorator.

In [ ]:
from smolagents import Tool
from langchain_core.vectorstores import VectorStore

class RetrieverTool(Tool):
    name = 'retriever'
    description = "Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        'query': {
            'type': 'string',
            'description': "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = 'string'

    def __init__(self, vectordb: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string."

        docs = self.vectordb.similarity_search(
            query,
            k=7,
        )

        return "\nRetrieved documents:\n" + "".join(
            [f"==== Document {str(i)} ====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

## Agents

The agent will need these arguments upon initialization:
- `tools`: a list of tools that the agent will be able to call.
- `model`: the LLM to power the agent.

Our `model` must be a callable that takes as input a list of messages and returns text. It also needs to accept a `stop_sequences` argument that indicates when to stop its generation. Here, we directly use the `HfApiModel` class to call the Inference API.

We will use [`meta-llama/Llama-3.1.70B-Instruct`](https://huggingface.co/meta-llama/Llama-3.1-70B-Instruct), served on HuggingFace's Inference API.

In [ ]:
from smolagents import HfApiModel, ToolCallingAgent

model = HfApiModel('meta-llama/Llama-3.1-70B-Instruct')

In [ ]:
retriever_tool = RetrieverTool(vectordb=vectordb)

agent = ToolCallingAgent(
    model=model,
    tools=[retriever_tool]
)

Since we initialize the agent as a `ReactJsonAgent`, it has been automatically given a default system prompt that tells the LLM engine to process step-by-step generate tool calls as JSON blobs.

When its `.run()` method is launched, the agent takes care of calling the LLM engine, parsing the tool call JSON blobs and executing these tool calls, all in a loopo that ends only when the final answer is provided.

In [ ]:
agent_output = agent.run("How can I push a model to the Hub?")

print('Final output:')
print(agent_output)

## Agentic RAG VS standard RAG

We will use  [`meta-llama/Meta-Llama-3-70B-Instruct`](https://huggingface.co/meta-llama/Meta-Llama-3.1-70B-Instruct) for evaluation and download [`m-ric/huggingface_doc_qa_eval`](https://huggingface.co/datasets/m-ric/huggingface_doc_qa_eval) dataset for evaluation.

In [ ]:
eval_dataset = datasets.load_dataset('m-ric/huggingface_doc_qa_eval', split='train')

Before running the test, we need to make the agent less verbose:

In [ ]:
import logging

agent.logger.setLevel(logging.WARNING)

In [ ]:
outputs_agentic_rag = []

for example in tqdm(eval_dataset):
    question = example['question']

    enhanced_question = f"""Using the information contained in your knowledge base, which you can access with the 'retriever' tool,
give a comprehensive answer to the question below.
Respond only to the question asked, response should be concise and relevant to the question.
If you cannot find information, do not give up and try calling your retriever again with different arguments!
Make sure to have covered the question completely by calling the retriever tool several times with semantically different queries.
Your queries should not be questions but affirmative form sentences: e.g. rather than "How do I load a model from the Hub in bf16?", query should be "load a model from the Hub bf16 weights".

Question:
{question}"""

    answer = agent.run(enhanced_question)

    print("=======================================================")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f'True answer: {example["answer"]}')

    results_agentic = {
        'question': question,
        'true_answer': example['answer'],
        'source_doc': example['source_doc'],
        'generated_answer': answer
    }

    outputs_agentic_rag.append(results_agentic)

In [ ]:
from huggingface_hub import InferenceClient

reader_llm = InferenceClient(model='meta-llama/Llama-3.1-70B-Instruct') # keep the same LLM engine

outputs_standard_rag = []

for example in tqdm(eval_dataset):
    question = example['question']
    context = retriever_tool(question)

        prompt = f"""Given the question and supporting documents below, give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If you cannot find information, do not give up and try calling your retriever again with different arguments!

Question:
{question}

{context}
"""

    messages = [{'role': 'user', 'content': prompt}]
    answer = reader_llm.chat_completion(messages).choices[0].message.content

    print("=======================================================")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f'True answer: {example["answer"]}')

    results_agentic = {
        "question": question,
        "true_answer": example["answer"],
        "source_doc": example["source_doc"],
        "generated_answer": answer,
    }
    outputs_standard_rag.append(results_agentic)

The evaluation prompt follows some of the best principles in the LLM-as-a-Judge examples.

In [ ]:
EVALUATION_PROMPT = """You are a fair evaluator language model.

You will be given an instruction, a response to evaluate, a reference answer that gets a score of 3, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 3. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 3}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.
5. Do not score conciseness: a correct answer that covers the question should receive max score, even if it contains additional useless information.

The instruction to evaluate:
{instruction}

Response to evaluate:
{response}

Reference Answer (Score 3):
{reference_answer}

Score Rubrics:
[Is the response complete, accurate, and factual based on the reference answer?]
Score 1: The response is completely incomplete, inaccurate, and/or not factual.
Score 2: The response is somewhat complete, accurate, and/or factual.
Score 3: The response is completely complete, accurate, and/or factual.

Feedback:"""

In [ ]:
from huggingface_hub import InferenceClient

evaluation_client = InferenceClient("meta-llama/Llama-3.1-70B-Instruct")

In [ ]:
import pandas as pd

results = {}

for system_type, outputs in [
    ('agentic', outputs_agentic_rag),
    ('standard', outputs_standard_rag)
]:
    for experiment in tqdm(outputs):
        eval_prompt = EVALUATION_PROMPT.format(
            instruction=experiment['question'],
            response=experiment['generated_answer'],
            reference_answer=experiment['true_answer']
        )

        messages = [
            {'role': 'system', 'content': "You are a fair evaluator language model."},
            {'role': 'user', 'content': eval_prompt}
        ]

        eval_result = evaluation_client.text_generation(eval_prompt, max_new_tokens=1000)

        try:
            feedback, score = [item.strip() for item in eval_result.split('[RESULT]')]
            experiment['eval_score_LLM_judge'] = score
            experiment['eval_feedback_LLM_judge'] = feedback
        except:
            print(f"Parsing failed - output was: {eval_result}")


    results[system_type] = pd.DataFrame.from_dict(outputs)
    results[system_type] = results[system_type].loc[~results[system_type]["generated_answer"].str.contains("Error")]

In [ ]:
# Give average score whenever scoring fails
DEFAULT_SCORE = 2

def fill_score(x):
    try:
        return int(x)
    except:
        return DEFAULT_SCORE


for system_type, outputs in [
    ('agentic', outputs_agentic_rag),
    ('standard', outputs_standard_rag)
]:
    results[system_type]['eval_score_LLM_judge_int'] = results[system_type]['eval_score_LLM_judge'].fillna(DEFAULT_SCORE).apply(fill_score)

    results[system_type]['eval_score_LLM_judge_int'] = (results[system_type]['eval_score_LLM_judge_int']) - 1) / 2

    print(f"Average score for {system_type} RAG: {results[system_type]['eval_score_LLM_judge_int'].mean()*100:.1f}%")